# convert_txt_to_csv

This notebook converts an IES results .txt file into a formatted csv file ready for analysis in Excel or other software.

Use ***Shift-Enter*** or ***Ctrl-Enter*** to run the code cells.

## Step 1: Choose the file to convert

The filename of the file to be converted is set in the cell below. Modify this as needed and then run the cell.

In [8]:
filename='MyWeather.txt'

## Step 2: Run the code below to convert the file

The cell below converts the file. The new file is saved as a .csv file The filename is the original filename in Step 1 with an additional '.csv' extension. The first 5 rows of the new file are displayed in the cell output in this notebook.

In [148]:
import pandas as pd
def convert_txt_to_csv(filename):
    try:
        df=pd.read_csv(filename,sep='\t',encoding = 'unicode_escape',header=[0,1,2])
        df.columns=pd.MultiIndex.from_arrays([df.columns.get_level_values(0),
                                              [x if not x.startswith('Unnamed') else 'NA' for x in df.columns.get_level_values(1)],
                                              df.columns.get_level_values(2)])
    except pd.errors.ParserError:
        df=pd.read_csv(filename,sep='\t',encoding = 'unicode_escape',header=[0,2])
        df.columns=pd.MultiIndex.from_arrays([df.columns.get_level_values(0),
                                              ['NA']*len(df.columns.get_level_values(0)),
                                              df.columns.get_level_values(1)])
    df.columns=df.columns.set_names(['variable_name', 'zone_name','file_name'])
    
    df[df.columns[0]]=df[df.columns[0]].fillna(method='ffill')
    df[df.columns[0]]=df[df.columns[0]].str[5:] + r'/2003'
    df[df.columns[1]]=df[df.columns[1]].replace(to_replace='24:00',value='00:00')
    df.insert(0,column='datetime',value=pd.to_datetime(df[df.columns[0]]+' '+df[df.columns[1]],format='%d/%b/%Y %H:%M'))
    df=df.drop(columns=[df.columns[1],df.columns[2]])
    df=df.set_index(df.columns[0])
    df.index=df.index.rename('datetime')
    
    new_filename=filename+'.csv'
    df.to_csv(new_filename)
    print('NEW FILENAME: ' + new_filename)
    return df

In [149]:
filename='MyResultsWeatherOnly.txt'
df=convert_txt_to_csv(filename)
df.head()

NEW FILENAME: MyResultsWeatherOnly.txt.csv


variable_name,Dry-bulb temperature (°C)
zone_name,NA
file_name,KEW.FWT
datetime,
2003-01-01 00:00:00,NaN
2003-01-01 01:00:00,4.8
2003-01-01 02:00:00,4.7
2003-01-01 03:00:00,4.3
2003-01-01 04:00:00,4.0


In [150]:
filename='MyResultsModelOnly.txt'
df=convert_txt_to_csv(filename)
df.head()

NEW FILENAME: MyResultsModelOnly.txt.csv


variable_name,Air temperature (°C)
zone_name,South Zone
file_name,Small_Office.aps
datetime,
2003-01-01 00:30:00,19.0
2003-01-01 01:30:00,19.0
2003-01-01 02:30:00,19.0
2003-01-01 03:30:00,19.0
2003-01-01 04:30:00,19.0


In [151]:
filename='MyResultsWeatherAndModel.txt'
df=convert_txt_to_csv(filename)
df.head()

NEW FILENAME: MyResultsWeatherAndModel.txt.csv


variable_name,Air temperature (°C),Dry-bulb temperature (°C)
zone_name,South Zone,NA
file_name,Small_Office.aps,KEW.FWT
datetime,,
2003-01-01 00:00:00,NaN,NaN
2003-01-01 00:30:00,19.0,NaN
2003-01-01 01:00:00,NaN,4.8
2003-01-01 01:30:00,19.0,NaN
2003-01-01 02:00:00,NaN,4.7


## Explanation

The weather data in IES VistaPro can be exported using the Save icon when viewing a table. To do this:

- Run an IES simulation
- Go to the VistaPro results module
- Select the weather data variables of interest
- View these variable in a table
- Click on the *save* icon to export this data as a .txt file.

The exported text file is not a user-firendly format. It has 2 header rows. The dates are recorded in a non-standard format (i.e. 'Wed, 01/Jan') and the dates only recorded at the first hour of each day rather than on each hourly row. The times include '24:00' rather than '00:00' to represent midnight. The data and time values are also seperate columns. These features make it difficult to directly use the .txt file for analysis.

This notebook opens the .txt file, modifies the data, and then saves a newly formatted version as a comma separated variable .csv file. The new file has a single header row and standard-format datetime column (which contains the date and the time for each row). The new .csv file can be opened in Excel or python for analysis and visualisation.


## Extra

To reload a csv file created by this notebook, the following can be used:

In [162]:
df=pd.read_csv('MyResultsWeatherOnly.txt.csv',header=[0,1,2],index_col=0,parse_dates=True)
df.head()

variable_name,Dry-bulb temperature (°C)
zone_name,NA
file_name,KEW.FWT
datetime,
2003-01-01 00:00:00,NaN
2003-01-01 01:00:00,4.8
2003-01-01 02:00:00,4.7
2003-01-01 03:00:00,4.3
2003-01-01 04:00:00,4.0
